In [10]:
sourceURL = "http://trafiopendata.97.fi/opendata/10072018_Ajoneuvojen_avoin_data_5_3.zip"
datadir = "./trafidata/"


# Haetaan aineisto Trafin webbisivulta ja puretaan zippi

import os
if not os.path.exists(datadir):
    os.makedirs(datadir)

if not os.path.isfile(datadir+"data.zip"):
    
    import urllib.request
    #import urllib
    urllib.request.urlretrieve (sourceURL, datadir+"data.zip")

if not os.path.isfile(datadir+"Ajoneuvojen avoin data 5.3.csv"):
    import zipfile
    zip_ref = zipfile.ZipFile(datadir+"data.zip", 'r')
    zip_ref.extractall(datadir)
    zip_ref.close()

# Luetaan aineisto Pandas dDataframeen... Paitsi että ei lueta, koska Big Data
import pandas as pd
df = pd.read_csv(datadir+"Ajoneuvojen avoin data 5.3.csv",sep=";",encoding="latin-1")

# RDD-tapa tehdä asioita Sparkissa. So last season...

import pyspark
sc = pyspark.SparkContext('local[*]')

# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

In [11]:
# Ladataan CSV Spark dataframeen

from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master("local")\
.appName("Data cleaning")\
.getOrCreate()

df = spark.read.format("csv")\
.option("header", "true")\
.option("delimiter", ";")\
.option("encoding", "ISO-8859-1")\
.load(datadir+"Ajoneuvojen avoin data 5.3.csv")

In [13]:
type(df)

pyspark.sql.dataframe.DataFrame

In [14]:
df.printSchema()

root
 |-- ajoneuvoluokka: string (nullable = true)
 |-- ensirekisterointipvm: string (nullable = true)
 |-- ajoneuvoryhma: string (nullable = true)
 |-- ajoneuvonkaytto: string (nullable = true)
 |-- variantti: string (nullable = true)
 |-- versio: string (nullable = true)
 |-- kayttoonottopvm: string (nullable = true)
 |-- vari: string (nullable = true)
 |-- ovienLukumaara: string (nullable = true)
 |-- korityyppi: string (nullable = true)
 |-- ohjaamotyyppi: string (nullable = true)
 |-- istumapaikkojenLkm: string (nullable = true)
 |-- omamassa: string (nullable = true)
 |-- teknSuurSallKokmassa: string (nullable = true)
 |-- tieliikSuurSallKokmassa: string (nullable = true)
 |-- ajonKokPituus: string (nullable = true)
 |-- ajonLeveys: string (nullable = true)
 |-- ajonKorkeus: string (nullable = true)
 |-- kayttovoima: string (nullable = true)
 |-- iskutilavuus: string (nullable = true)
 |-- suurinNettoteho: string (nullable = true)
 |-- sylintereidenLkm: string (nullable = true)
 

In [16]:
df.head(5)

[Row(ajoneuvoluokka='MUU', ensirekisterointipvm=None, ajoneuvoryhma='21', ajoneuvonkaytto='01', variantti=None, versio=None, kayttoonottopvm='19670000', vari=None, ovienLukumaara=None, korityyppi=None, ohjaamotyyppi=None, istumapaikkojenLkm='1', omamassa='210', teknSuurSallKokmassa=None, tieliikSuurSallKokmassa=None, ajonKokPituus=None, ajonLeveys=None, ajonKorkeus=None, kayttovoima='01', iskutilavuus='590', suurinNettoteho=None, sylintereidenLkm=None, ahdin=None, sahkohybridi=None, merkkiSelvakielinen='BMW', mallimerkinta='R60/590', vaihteisto=None, vaihteidenLkm=None, kaupallinenNimi=None, voimanvalJaTehostamistapa=None, tyyppihyvaksyntanro=None, yksittaisKayttovoima='01', kunta='049', Co2=None, matkamittarilukema=None, alue='027', valmistenumero2=None, jarnro='1'),
 Row(ajoneuvoluokka='M1', ensirekisterointipvm='1997-01-10', ajoneuvoryhma=None, ajoneuvonkaytto='01', variantti=None, versio=None, kayttoonottopvm='19970110', vari='5', ovienLukumaara='5', korityyppi='AC', ohjaamotyyppi=

In [18]:
df.filter(df.ensirekisterointipvm >= "2017-01-01").count()

357257

In [43]:
df.filter((df.ensirekisterointipvm >= "2018-01-01") & ((df.ajoneuvoluokka == "M1") | (df.ajoneuvoluokka == "M1G"))).count()

85607

In [42]:
df.filter((df.ensirekisterointipvm >= "2018-01-01") & ((df.ajoneuvoluokka == "M1") | (df.ajoneuvoluokka == "M1G"))).groupby("kayttovoima", "sahkohybridi").count().orderBy("kayttovoima").collect()

[Row(kayttovoima='01', sahkohybridi='false', count=46887),
 Row(kayttovoima='01', sahkohybridi='true', count=10334),
 Row(kayttovoima='01', sahkohybridi=None, count=49),
 Row(kayttovoima='02', sahkohybridi='true', count=285),
 Row(kayttovoima='02', sahkohybridi='false', count=26156),
 Row(kayttovoima='02', sahkohybridi=None, count=28),
 Row(kayttovoima='04', sahkohybridi='false', count=426),
 Row(kayttovoima='04', sahkohybridi='true', count=2),
 Row(kayttovoima='11', sahkohybridi='false', count=1),
 Row(kayttovoima='13', sahkohybridi='false', count=167),
 Row(kayttovoima='38', sahkohybridi='false', count=1122),
 Row(kayttovoima='38', sahkohybridi=None, count=1),
 Row(kayttovoima='39', sahkohybridi='false', count=4),
 Row(kayttovoima='39', sahkohybridi='true', count=26),
 Row(kayttovoima='40', sahkohybridi='false', count=114),
 Row(kayttovoima='41', sahkohybridi='false', count=3),
 Row(kayttovoima='42', sahkohybridi='false', count=1),
 Row(kayttovoima='44', sahkohybridi='true', count=1)

In [21]:
df.filter(df.ensirekisterointipvm > "2017-01-01").count().orderBy("ajoneuvoluokka", "kayttovoima").show()

AttributeError: 'int' object has no attribute 'orderBy'